In [1]:
import numpy as np
import csv
 
csv_file = "/kaggle/input/kuhar-ex-feat/kuhar_img84_X_1024features_irnv2.csv"
 
data_listirnv2 = []
with open(csv_file, "r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data_listirnv2.append(row)
print(len(data_listirnv2))
print(type(data_listirnv2))

15571
<class 'list'>


In [2]:
data_listirnv2=data_listirnv2[1:15571]

In [3]:
import numpy as np
irnv2=np.asarray(data_listirnv2)
irnv2.shape

(15570, 1024)

In [4]:
import numpy as np
import csv
 
csv_file = "/kaggle/input/kuhar-ex-feat/kuhar_img84_X_2304features_ex.csv"
 
data_listex = []
with open(csv_file, "r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data_listex.append(row)
print(len(data_listex))
print(type(data_listex))

15571
<class 'list'>


In [5]:
data_listex=data_listex[1:15571]

In [6]:
import numpy as np
ex=np.asarray(data_listex)
ex.shape

(15570, 2304)

In [7]:
import numpy as np
import csv
 
csv_file = "/kaggle/input/kuhar-ex-feat/kuhar_img84_X_2304features_env2b1.csv"
 
data_listenv2b1 = []
with open(csv_file, "r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data_listenv2b1.append(row)
print(len(data_listenv2b1))
print(type(data_listenv2b1))

15571
<class 'list'>


In [8]:
data_listenv2b1=data_listenv2b1[1:15571]

In [9]:
import numpy as np
env2b1=np.asarray(data_listenv2b1)
env2b1.shape

(15570, 2304)

In [10]:
train_feat=np.concatenate([irnv2,ex,env2b1],axis=1)

In [11]:
train_feat.shape

(15570, 5632)

In [12]:
import pandas as pd
df=pd.DataFrame(train_feat)
df.to_csv('merged_kuhar_X_feature_img84.csv',index=False)

In [13]:
import numpy as np
import csv
 
csv_file = "/kaggle/input/kuhar-ex-feat/kuhar_img84_Y_1024features_irnv2.csv"
 
data_listirnv2 = []
with open(csv_file, "r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data_listirnv2.append(row)
print(len(data_listirnv2))
print(type(data_listirnv2))

15571
<class 'list'>


In [14]:
data_listirnv2=data_listirnv2[1:15571]

In [15]:
import numpy as np
Y=np.asarray(data_listirnv2)
Y.shape

(15570, 1)

In [16]:
import pandas as pd
df=pd.DataFrame(Y)
df.to_csv('Y_kuhar_img84.csv',index=False)

In [17]:
path='/kaggle/working/Y_kuhar_img84.csv'
df_y = pd.read_csv(path,nrows=15570)

In [18]:
df_y.head()

,0
0,1
1,1
2,1
3,1
4,1


In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
def split(df,label):
    X_tr, X_te, Y_tr, Y_te = train_test_split(df, label, test_size=0.25, random_state=42)
    return X_tr, X_te, Y_tr, Y_te

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
classifiers = ['LinearSVM', 'RadialSVM', 
               'Logistic',  'RandomForest', 
               'KNeighbors']

models = [svm.SVC(kernel='linear'),
          svm.SVC(kernel='rbf'),
          LogisticRegression(max_iter = 1000),
          RandomForestClassifier(n_estimators=200, random_state=0),
          #AdaBoostClassifier(random_state = 0),
          #DecisionTreeClassifier(random_state=0),
          KNeighborsClassifier(),
          #GradientBoostingClassifier(random_state=0)
         ]


def acc_score(df,label):
    Score = pd.DataFrame({"Classifier":classifiers})
    j = 0
    acc = []
    X_train,X_test,Y_train,Y_test = split(df,label)
    for i in models:
        model = i
        model.fit(X_train,Y_train)
        predictions = model.predict(X_test)
        acc.append(accuracy_score(Y_test,predictions))
        j = j+1     
    Score["Accuracy"] = acc
    Score.sort_values(by="Accuracy", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score

def plot(score,x,y,c = "b"):
    gen = [1,2,3,4,5,6,7]
    plt.figure(figsize=(6,4))
    ax = sns.pointplot(x=gen, y=score,color = c )
    ax.set(xlabel="Generation", ylabel="Accuracy")
    ax.set(ylim=(x,y))

In [21]:
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=bool)     
        chromosome[:int(0.3*n_feat)]=False             
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population


def fitness_score(population):
    scores = []
    for chromosome in population:
        logmodel.fit(X_train.iloc[:,chromosome],Y_train)         
        predictions = logmodel.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(Y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)                                    
    return list(scores[inds][::-1]), list(population[inds,:][::-1]) 


def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen


def crossover(pop_after_sel):
    pop_nextgen = pop_after_sel
    for i in range(0,len(pop_after_sel),2):
        new_par = []
        child_1 , child_2 = pop_nextgen[i] , pop_nextgen[i+1]
        new_par = np.concatenate((child_1[:len(child_1)//2],child_2[len(child_1)//2:]))
        pop_nextgen.append(new_par)
    return pop_nextgen

def mutation(pop_after_cross,mutation_rate,n_feat):   
    mutation_range = int(mutation_rate*n_feat)
    pop_next_gen = []
    for n in range(0,len(pop_after_cross)):
        chromo = pop_after_cross[n]
        rand_posi = [] 
        for i in range(0,mutation_range):
            pos = randint(0,n_feat-1)
            rand_posi.append(pos)
        for j in rand_posi:
            chromo[j] = not chromo[j]  
        pop_next_gen.append(chromo)
    return pop_next_gen

def generations(df,label,size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, Y_train, Y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print('Best score in generation',i+1,':',scores[:1])  #2
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate,n_feat)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

In [19]:
import pandas as pd
path='/kaggle/working/merged_kuhar_X_feature_img84.csv'
df_total = pd.read_csv(path,nrows=15570)

In [20]:
df_total.head()

,0,1,2,3,4,5,6,7,8,9,...,5622,5623,5624,5625,5626,5627,5628,5629,5630,5631
0,14978.139,0.0,21749.326,9588.747,0.0,20502.270,359.12390,15773.608,5690.9520,32306.984,...,0.0,1830.1443,0.0,1399.1044,0.0,1102.1595,2624.45120,1426.47730,998.84485,0.0
1,15384.276,0.0,22381.244,9601.602,0.0,20993.130,162.04968,16177.826,5994.6680,32680.324,...,0.0,1491.1224,0.0,0.0000,0.0,0.0000,540.22864,110.28999,0.00000,0.0
2,15418.544,0.0,22653.729,9416.146,0.0,21118.074,239.07507,16202.858,5901.4210,32757.078,...,0.0,1049.0372,0.0,0.0000,0.0,0.0000,0.00000,0.00000,0.00000,0.0
3,15477.536,0.0,22657.055,9734.257,0.0,21276.033,278.38074,16289.794,6022.4033,33177.350,...,0.0,1488.5052,0.0,0.0000,0.0,0.0000,547.72860,67.67592,0.00000,0.0
4,15492.198,0.0,22610.068,9591.143,0.0,21178.640,225.74109,16266.511,5963.9424,32977.055,...,0.0,1026.7000,0.0,0.0000,0.0,0.0000,0.00000,0.00000,0.00000,0.0


In [21]:
for column in df_total.columns:
    if df_total[column].dtype == "float64":
        df_total[column]=pd.to_numeric(df_total[column], downcast="float")
    if df_total[column].dtype == "int64":
        df_total[column]=pd.to_numeric(df_total[column], downcast="integer")

In [22]:
df_y.head()

,0
0,1
1,1
2,1
3,1
4,1


In [23]:
for column in df_y.columns:
    if df_y[column].dtype == "float64":
        df_y[column]=pd.to_numeric(df_y[column], downcast="float")
    if df_y[column].dtype == "int64":
        df_y[column]=pd.to_numeric(df_y[column], downcast="integer")

In [25]:
score1 = acc_score(df_total,df_y)
score1

,Classifier,Accuracy
0,Logistic,0.835345
1,LinearSVM,0.824814
2,RandomForest,0.776008
3,KNeighbors,0.690984
4,RadialSVM,0.606216


In [25]:
# Load libraries
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import warnings
warnings.filterwarnings("ignore")

# Load data
df = pd.read_csv('/kaggle/working/merged_kuhar_X_feature_img84.csv')

y = pd.read_csv('/kaggle/working/Y_kuhar_img84.csv')


# Create features and target
X = df
#y = df['class']


# Select K best features
k_features = 1500
k_best_features = SelectKBest(chi2, k = k_features).fit(X, y)
best_features = df[X.columns[k_best_features.get_support()]]
print('Selected top', k_features, 'features: \n', best_features.columns, '\n')


#print('max ',ind)
X_train, X_test, y_train, y_test = train_test_split(best_features, y, test_size = 0.25)
clf = SVC(kernel='linear') 
clf.fit(X_train, y_train)
accuracy = clf.score(X_test,y_test)

print('Accuracy: ',accuracy)

target_names=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18']
y_predict = clf.predict(X_test)

print(classification_report(y_test, y_predict, target_names=target_names))

best_features ['class'] = y 
best_features.to_csv(r'chi-filter.csv', index = False )


Selected top 1500 features: 
 Index(['552', '793', '837', '988', '1024', '1025', '1027', '1033', '1037',
       '1041',
       ...
       '3321', '3322', '3323', '3324', '3325', '3463', '3529', '5081', '5124',
       '5601'],
      dtype='object', length=1500) 

Accuracy:  0.8147957873105575
              precision    recall  f1-score   support

           1       0.52      0.61      0.56       360
           2       0.61      0.52      0.56       384
           3       0.71      0.73      0.72       319
           4       0.93      0.94      0.93       360
           5       0.95      0.96      0.95       410
           6       0.68      0.68      0.68       326
           7       0.95      0.93      0.94       340
           8       0.90      0.91      0.91       248
           9       0.98      0.96      0.97       110
          10       0.95      0.91      0.93        92
          11       0.92      0.91      0.91       165
          12       0.91      0.84      0.87       169
    